In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import utils

In [2]:
rows = ['Zonal_Winds', 'Meridional_Winds', 'Humidity', 'Air_Temp', 'Sea_Surface_Temp']
train_data = pd.read_csv('../../Cleaning-the-data/tao-cleaned.csv')[rows]

train_data = (train_data - train_data.mean())/train_data.std()
train_data = train_data.values[:17000]
train_data.shape

(17000, 5)

In [3]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(5,50, 3)
        self.lstm2 = nn.LSTM(50,1, 2)
        
 
    def forward(self, seq):
        lstm1_out, _ = self.lstm1(seq, None)
        lstm2_out, _ = self.lstm2(lstm1_out, None)
        return lstm2_out.squeeze(1)

In [4]:
seq = Sequence()
test_data = torch.Tensor(train_data[:10]).view(-1,1,5)
test_label = torch.Tensor(train_data[:10][:,-1]).view(-1,1)

print(test_label)
print(seq(test_data))

tensor([[-0.7540],
        [-0.8920],
        [-1.2446],
        [-1.7403],
        [-2.3127],
        [-2.0827],
        [-1.7250],
        [-1.8272],
        [-1.7761],
        [-0.9073]])
tensor([[-0.1510],
        [-0.1637],
        [-0.1620],
        [-0.1585],
        [-0.1557],
        [-0.1538],
        [-0.1526],
        [-0.1519],
        [-0.1514],
        [-0.1511]], grad_fn=<SqueezeBackward1>)


In [5]:
criterion = nn.MSELoss()
lr = 0.01
optimizer = optim.Adam(seq.parameters(), lr=lr)
seq_size = 100

In [6]:
for epoch in range(200):
    running_loss = 0
    running_error = 0
    num_batches = 0
    for i in range(0, 15000, seq_size):
        data = train_data[i:i+seq_size]
        xs = torch.Tensor(data[:-1]).view(-1, 1, 5)
        ys = torch.Tensor(data[1:][:,-1]).view(-1,1)
        
        lstm_out = seq(xs)
        loss = criterion(lstm_out.view(1, -1), ys.view(1, -1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute some stats:
        
        num_batches+=1
        
        with torch.no_grad():
            running_loss += loss.item()
            error = utils.get_error(lstm_out.view(1, -1), ys.view(1, -1))
            running_error += error 

    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    if epoch % 5 == 0:
        total_loss = running_loss/num_batches
        total_error = running_error/num_batches
        print('epoch=',epoch, '\t loss=', total_loss, '\t error={:2.2%}'.format(total_error) ,'percent')

epoch= 0 	 loss= 1.7649327496439218 	 error=69.97% percent
epoch= 5 	 loss= 1.2651004884764552 	 error=66.42% percent
epoch= 10 	 loss= 1.0834984723913172 	 error=48.69% percent
epoch= 15 	 loss= 1.0611894478059063 	 error=47.70% percent
epoch= 20 	 loss= 1.0551796595402994 	 error=47.29% percent
epoch= 25 	 loss= 1.0517618568963372 	 error=47.19% percent
epoch= 30 	 loss= 1.0519355270534287 	 error=47.19% percent
epoch= 35 	 loss= 1.0483842568685455 	 error=47.04% percent
epoch= 40 	 loss= 1.0478957282596577 	 error=47.01% percent
epoch= 45 	 loss= 1.046509543004989 	 error=46.90% percent
epoch= 50 	 loss= 1.0464610926318954 	 error=46.89% percent
epoch= 55 	 loss= 1.0449518260036712 	 error=46.84% percent
epoch= 60 	 loss= 1.0451748041643683 	 error=46.87% percent
epoch= 65 	 loss= 1.0443355769877478 	 error=46.79% percent
epoch= 70 	 loss= 1.0451134440626872 	 error=46.88% percent
epoch= 75 	 loss= 1.0433048350331955 	 error=46.76% percent
epoch= 80 	 loss= 1.0427989541193528 	 erro

KeyboardInterrupt: 